<a href="https://colab.research.google.com/github/skyqi/LangChain/blob/main/ConversationBufferWindowMemory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ConversationBufferWindowMemory
ConversationBufferWindowMemory是一个用于存储和访问对话历史记录的内存类型。它保存了对话过程中的一系列交互，但只使用最近K个交互。防止缓冲区变得过大。

In [ ]:
import os
import getpass

if not os.environ["ZHIPUAI_API_KEY"]:
  os.environ["ZHIPUAI_API_KEY"] = getpass.getpass()

In [ ]:
!pip install --upgrade --quiet  langchain-google-genai pillow
!pip install langchain langchain_community
!pip install --upgrade httpx httpx-sse PyJWT

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 13.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.7 MB/s eta 0:00:00

In [ ]:
from langchain_community.chat_models import ChatZhipuAI

from langchain_community.chat_message_histories import ChatMessageHistory

#from langchain_core.chat_history import BaseChatMessageHistory
#from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.chains import ConversationChain

llm = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)

In [ ]:
# 修改 store 字典以存储不同 session_id 的 memory 实例
store = {}

def get_conversation_memory(session_id: str) -> ConversationBufferWindowMemory:
    if session_id not in store:
        store[session_id] = ConversationBufferWindowMemory(k=3)
    return store[session_id]

# 使用 session_id 创建或获取 ConversationBufferMemory 实例
def create_conversation(llm, session_id: str) -> ConversationChain:
    memory = get_conversation_memory(session_id)
    return ConversationChain(llm=llm, memory=memory, verbose=True)

# 示例用法，第一问
conversation = create_conversation(llm, "xinhu_session")
conversation.predict(input="你好, 我的名字叫小虎")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好, 我的名字叫小虎
AI:

> Finished chain.


'你好，小虎！很高兴认识你。今天想和我聊些什么呢？有没有什么问题或者话题是你特别感兴趣的？我会尽力提供有帮助的回答哦！'

In [ ]:
# 示例用法，第二问
conversation = create_conversation(llm, "xinhu_session")
conversation.predict(input="我的老大是黑金")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好, 我的名字叫小虎
AI: 你好，小虎！很高兴认识你。今天想和我聊些什么呢？有没有什么问题或者话题是你特别感兴趣的？我会尽力提供有帮助的回答哦！
Human: 我的老大是黑金
AI:

> Finished chain.


'哦，黑金是一个很有力量的名字呢！听起来你的老大应该是个很有个性的人。那么，你是想聊聊关于你老大的事情，还是有什么其他的问题或者想聊的话题？如果有关黑金的故事或者你想知道的信息，尽管告诉我。'

这去掉了很多的代码调试...

# 问一个三次过后的记录
通过以上五次问，只取最后三条聊天记录

In [ ]:
#一共问了5条记录，反过来问第5条记录。结果AI不知道了
conversation = create_conversation(llm, "xinhu_session")
conversation.predict(input="我的老大是谁")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 我的老大的女朋友叫林妹
AI: 嗯，你之前提到过你老大的女朋友叫林妹。如果这个话题对你很重要，我们可以继续聊。你想分享关于林妹的哪些事情呢？或者，你对恋爱关系有什么疑问，需要我帮忙解答的吗？如果还有其他的事情你想知道或者讨论，也可以直接告诉我。
Human: 林妹有一条狗叫翘嘴
AI: 哈哈，翘嘴这个名字真有趣，给人一种活泼可爱的感觉。看来林妹养的狗也有着自己的个性。如果你想分享更多关于翘嘴的故事，或者讨论关于宠物的话题，比如如何照顾宠物、宠物与主人之间的关系等，我都很乐意听你讲述并给予我的看法。当然，如果你有其他问题或者想要了解的信息，也可以继续提问。
Human: 翘嘴每天早上爱吃火腿肠，林妹早上喜欢喝豆浆
AI: 翘嘴喜欢早上吃火腿肠，而林妹则喜欢喝豆浆，这两者的早餐选择反映了它们各自独特的口味。宠物的饮食习惯和主人的喜好有时候会互相影响，形成一种温馨的生活画面。如果你想了解更多关于宠物的营养建议，或者是关于健康早餐的想法，我都可以提供帮助。另外，不同地区的早餐文化也很有趣，比如在中国的不同地方，早餐的选择会有很大的差异，我们可以聊聊这些话题，或者如果你有其他兴趣点，也可以继续分享。
Human: 我的老大是谁
AI:

> Finished chain.


'关于这个问题，因为你没有提供具体的上下文信息，我无法准确地知道你的“老大”是指谁。在中文里，“老大”可以指很多不同的人或事物，比如家庭成员、朋友、工作中的上司，或者是某个团体、组织的领导者。如果你能提供更多的信息，我会很乐意帮你分析并给出我的理解。如果这个问题是开玩笑或者是在特定情境下的一个梗，也请告诉我，我们可以一起探讨这个话题。'

In [ ]:
conversation = create_conversation(llm, "xinhu_session")
conversation.predict(input="林妹的狗叫什么")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 林妹有一条狗叫翘嘴
AI: 哈哈，翘嘴这个名字真有趣，给人一种活泼可爱的感觉。看来林妹养的狗也有着自己的个性。如果你想分享更多关于翘嘴的故事，或者讨论关于宠物的话题，比如如何照顾宠物、宠物与主人之间的关系等，我都很乐意听你讲述并给予我的看法。当然，如果你有其他问题或者想要了解的信息，也可以继续提问。
Human: 翘嘴每天早上爱吃火腿肠，林妹早上喜欢喝豆浆
AI: 翘嘴喜欢早上吃火腿肠，而林妹则喜欢喝豆浆，这两者的早餐选择反映了它们各自独特的口味。宠物的饮食习惯和主人的喜好有时候会互相影响，形成一种温馨的生活画面。如果你想了解更多关于宠物的营养建议，或者是关于健康早餐的想法，我都可以提供帮助。另外，不同地区的早餐文化也很有趣，比如在中国的不同地方，早餐的选择会有很大的差异，我们可以聊聊这些话题，或者如果你有其他兴趣点，也可以继续分享。
Human: 我的老大是谁
AI: 关于这个问题，因为你没有提供具体的上下文信息，我无法准确地知道你的“老大”是指谁。在中文里，“老大”可以指很多不同的人或事物，比如家庭成员、朋友、工作中的上司，或者是某个团体、组织的领导者。如果你能提供更多的信息，我会很乐意帮你分析并给出我的理解。如果这个问题是开玩笑或者是在特定情境下的一个梗，也请告诉我，我们可以一起探讨这个话题。
Human: 林妹的狗叫什么
AI:

> Finished chain.


'林妹的狗叫翘嘴。这个名字很特别，让人印象深刻。如果还有其他关于翘嘴的问题，或者想要讨论其他话题，请随时告诉我。'

In [ ]:
#输入全部的聊天记录
history = conversation.memory.chat_memory.messages
history

[HumanMessage(content='你好, 我的名字叫小虎'),
 AIMessage(content='你好，小虎！很高兴认识你。今天想和我聊些什么呢？有没有什么问题或者话题是你特别感兴趣的？我会尽力提供有帮助的回答哦！'),
 HumanMessage(content='我的老大是黑金'),
 AIMessage(content='哦，黑金是一个很有力量的名字呢！听起来你的老大应该是个很有个性的人。那么，你是想聊聊关于你老大的事情，还是有什么其他的问题或者想聊的话题？如果有关黑金的故事或者你想知道的信息，尽管告诉我。'),
 HumanMessage(content='我的老大的女朋友叫林妹'),
 AIMessage(content='林妹，这个名字听起来很亲切。看来你的老大和他的女朋友之间的关系对你来说很重要。如果你想分享更多关于他们的故事，或者有什么关于恋爱关系的问题，我在这里都可以帮你解答。当然，如果你有其他任何问题，比如生活、文化、科技等方面的，也可以继续提问。我们可以一起探讨各种话题。'),
 HumanMessage(content='我的老大的女朋友叫林妹'),
 AIMessage(content='嗯，你之前提到过你老大的女朋友叫林妹。如果这个话题对你很重要，我们可以继续聊。你想分享关于林妹的哪些事情呢？或者，你对恋爱关系有什么疑问，需要我帮忙解答的吗？如果还有其他的事情你想知道或者讨论，也可以直接告诉我。'),
 HumanMessage(content='林妹有一条狗叫翘嘴'),
 AIMessage(content='哈哈，翘嘴这个名字真有趣，给人一种活泼可爱的感觉。看来林妹养的狗也有着自己的个性。如果你想分享更多关于翘嘴的故事，或者讨论关于宠物的话题，比如如何照顾宠物、宠物与主人之间的关系等，我都很乐意听你讲述并给予我的看法。当然，如果你有其他问题或者想要了解的信息，也可以继续提问。'),
 HumanMessage(content='翘嘴每天早上爱吃火腿肠，林妹早上喜欢喝豆浆'),
 AIMessage(content='翘嘴喜欢早上吃火腿肠，而林妹则喜欢喝豆浆，这两者的早餐选择反映了它们各自独特的口味。宠物的饮食习惯和主人的喜好有时候会互相影响，形成一种温馨的生活画面。如果你想了解更多关于宠物的营养建议，或者是关于健康早

In [ ]:
#输出全部的聊天记录
conversation = create_conversation(llm, "heijing_session")
history = conversation.memory.chat_memory.messages
history

[HumanMessage(content='你好, 我的名字是什么'),
 AIMessage(content='你好！根据我们的对话历史，我没有存储你的个人信息，包括你的名字。所以，不好意思，我不知道你的名字是什么。如果你愿意告诉我，我会很高兴的。'),
 HumanMessage(content='林妹的狗叫什么'),
 AIMessage(content='林妹的狗的名字在我们的对话历史中没有提到过，所以我也不知道。如果你能提供更多的上下文或者它是某个公共知识中的信息，我可能能够帮助你找到答案。')]

In [ ]:
conversation = create_conversation(llm, "xinhu_session")
history = conversation.memory.chat_memory.messages
history

[HumanMessage(content='你好, 我的名字叫小虎'),
 AIMessage(content='你好，小虎！很高兴认识你。今天想和我聊些什么呢？有没有什么问题或者话题是你特别感兴趣的？我会尽力提供有帮助的回答哦！'),
 HumanMessage(content='我的老大是黑金'),
 AIMessage(content='哦，黑金是一个很有力量的名字呢！听起来你的老大应该是个很有个性的人。那么，你是想聊聊关于你老大的事情，还是有什么其他的问题或者想聊的话题？如果有关黑金的故事或者你想知道的信息，尽管告诉我。'),
 HumanMessage(content='我的老大的女朋友叫林妹'),
 AIMessage(content='林妹，这个名字听起来很亲切。看来你的老大和他的女朋友之间的关系对你来说很重要。如果你想分享更多关于他们的故事，或者有什么关于恋爱关系的问题，我在这里都可以帮你解答。当然，如果你有其他任何问题，比如生活、文化、科技等方面的，也可以继续提问。我们可以一起探讨各种话题。'),
 HumanMessage(content='我的老大的女朋友叫林妹'),
 AIMessage(content='嗯，你之前提到过你老大的女朋友叫林妹。如果这个话题对你很重要，我们可以继续聊。你想分享关于林妹的哪些事情呢？或者，你对恋爱关系有什么疑问，需要我帮忙解答的吗？如果还有其他的事情你想知道或者讨论，也可以直接告诉我。'),
 HumanMessage(content='林妹有一条狗叫翘嘴'),
 AIMessage(content='哈哈，翘嘴这个名字真有趣，给人一种活泼可爱的感觉。看来林妹养的狗也有着自己的个性。如果你想分享更多关于翘嘴的故事，或者讨论关于宠物的话题，比如如何照顾宠物、宠物与主人之间的关系等，我都很乐意听你讲述并给予我的看法。当然，如果你有其他问题或者想要了解的信息，也可以继续提问。'),
 HumanMessage(content='翘嘴每天早上爱吃火腿肠，林妹早上喜欢喝豆浆'),
 AIMessage(content='翘嘴喜欢早上吃火腿肠，而林妹则喜欢喝豆浆，这两者的早餐选择反映了它们各自独特的口味。宠物的饮食习惯和主人的喜好有时候会互相影响，形成一种温馨的生活画面。如果你想了解更多关于宠物的营养建议，或者是关于健康早

# 打印历史记录


In [ ]:
#打印xinhu_session的历史记录
conversation = create_conversation(llm, "xinhu_session")
history = conversation.memory.chat_memory.messages
history

[HumanMessage(content='你好, 我的名字叫小虎'),
 AIMessage(content='你好，小虎！很高兴认识你。今天想和我聊些什么呢？有没有什么问题或者话题是你特别感兴趣的？我会尽力提供有帮助的回答哦！'),
 HumanMessage(content='我的老大是黑金'),
 AIMessage(content='哦，黑金是一个很有力量的名字呢！听起来你的老大应该是个很有个性的人。那么，你是想聊聊关于你老大的事情，还是有什么其他的问题或者想聊的话题？如果有关黑金的故事或者你想知道的信息，尽管告诉我。'),
 HumanMessage(content='我的老大的女朋友叫林妹'),
 AIMessage(content='林妹，这个名字听起来很亲切。看来你的老大和他的女朋友之间的关系对你来说很重要。如果你想分享更多关于他们的故事，或者有什么关于恋爱关系的问题，我在这里都可以帮你解答。当然，如果你有其他任何问题，比如生活、文化、科技等方面的，也可以继续提问。我们可以一起探讨各种话题。'),
 HumanMessage(content='我的老大的女朋友叫林妹'),
 AIMessage(content='嗯，你之前提到过你老大的女朋友叫林妹。如果这个话题对你很重要，我们可以继续聊。你想分享关于林妹的哪些事情呢？或者，你对恋爱关系有什么疑问，需要我帮忙解答的吗？如果还有其他的事情你想知道或者讨论，也可以直接告诉我。'),
 HumanMessage(content='林妹有一条狗叫翘嘴'),
 AIMessage(content='哈哈，翘嘴这个名字真有趣，给人一种活泼可爱的感觉。看来林妹养的狗也有着自己的个性。如果你想分享更多关于翘嘴的故事，或者讨论关于宠物的话题，比如如何照顾宠物、宠物与主人之间的关系等，我都很乐意听你讲述并给予我的看法。当然，如果你有其他问题或者想要了解的信息，也可以继续提问。'),
 HumanMessage(content='翘嘴每天早上爱吃火腿肠，林妹早上喜欢喝豆浆'),
 AIMessage(content='翘嘴喜欢早上吃火腿肠，而林妹则喜欢喝豆浆，这两者的早餐选择反映了它们各自独特的口味。宠物的饮食习惯和主人的喜好有时候会互相影响，形成一种温馨的生活画面。如果你想了解更多关于宠物的营养建议，或者是关于健康早

In [ ]:
#打印heijing_session的历史记录
conversation = create_conversation(llm, "heijing_session")
history = conversation.memory.chat_memory.messages
history

[HumanMessage(content='你好, 我的名字是什么'),
 AIMessage(content='你好！根据我们的对话历史，我没有存储你的个人信息，包括你的名字。所以，不好意思，我不知道你的名字是什么。如果你愿意告诉我，我会很高兴的。'),
 HumanMessage(content='林妹的狗叫什么'),
 AIMessage(content='林妹的狗的名字在我们的对话历史中没有提到过，所以我也不知道。如果你能提供更多的上下文或者它是某个公共知识中的信息，我可能能够帮助你找到答案。'),
 HumanMessage(content='我叫黑金'),
 AIMessage(content='你好，黑金！很高兴你告诉我你的名字。有什么我可以帮你的吗？如果你有任何问题或者需要帮助，随时可以问我。'),
 HumanMessage(content='我的女朋友叫林妹'),
 AIMessage(content='明白了，黑金。那么，林妹是你的女朋友。你们俩有没有共同养的宠物呢？如果你之前提到的林妹的狗是你们共同的宠物，我还是没能知道它的名字。如果你愿意分享，我很乐意听你讲述。'),
 HumanMessage(content='我还有一个女性朋友叫小敏'),
 AIMessage(content='哦，好的，黑金。你有一个女朋友叫林妹，还有一个女性朋友叫小敏。人际关系网挺丰富的嘛！如果你想聊聊关于她们的事情，或者有其他任何问题，我都在这里听着呢。不管是生活中的小事，还是你想要了解的信息，都可以告诉我。'),
 HumanMessage(content='小敏住在南方'),
 AIMessage(content='原来如此，黑金。小敏住在南方啊？中国的南方地区很美，气候宜人，有很多风景胜地和独特的地方文化。不知道小敏是哪个城市的呢？如果你愿意分享更多关于小敏或者你们之间的趣事，我很乐意听你讲述。同时，如果你需要关于南方旅游、文化或者其他方面的信息，也可以问我。')]

# 总结

1.   只知道最后3个消息。流水记录的最后几个消息
2.   没有总结
3.   可以分session_id提问

